In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
import os, sys
from google.colab import drive
drive.mount('/gdrive')
sys.path.insert(0, "/content/gdrive/My Drive/CS 189/hw/hw6-collabtry/")
%cd /gdrive/My\ Drive/CS\ 189/hw/hw6-collabtry/

In [0]:
import numpy as np
from torchsummary import summary
import pandas as pd
from sklearn.metrics import confusion_matrix
import pickle
import torch

In [0]:
!pip install torchsummary

In [0]:
#https://github.com/Luolc/AdaBound
!pip install adabound

In [0]:
np.random.seed(10)

In [0]:
models_data = []

In [0]:
%matplotlib inline
import time
import pylab as pl
from IPython import display


def plot_kaggleloss(arch,train,val,status=None,extra=None):
    title = arch
    pl.figure().legends = []
    if status:
      a,b,c,d = status
      title += ', Epoch [{}/{}], Step [{}/{}]'.format(a,b,c,d)
    if len(train) > 0:
      mt = sum(train)/len(train)
      pl.plot(train, 'r', label="train")
      title += ", Avg Train Loss: "+str(round(mt,4))
    if len(val) > 0:
      mv = sum(val)/len(val)
      pl.plot(val, 'b', label="validation")
      title+=", Avg Val Loss: "+str(round(mv,4))
    if extra is not None:
      title+=", Current Val Acc: "+str(round(extra,4))
    pl.title(title)
    pl.xlabel("steps")
    pl.ylabel("loss")
    pl.legend()
    display.clear_output(wait=True)
    display.display(pl.gcf())
    time.sleep(1.0)
    
def data_track(data=None,save=True):
    if save and len(data) > 0:
        with open("datakaggle.txt", "wb") as file:   #Pickling
            pickle.dump(data, file)
        torch.cuda.empty_cache()
    else:
        torch.cuda.empty_cache()
        with open("datakaggle.txt", "rb") as file:   # Unpickling
            return pickle.load(file)

In [0]:
import torch
from torch.utils import data
import pandas as pd
import random
import json
import numpy as np
from skimage import io, transform
from PIL import Image

class Mds189(data.Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, label_file, loader, transform):
        'Initialization'
        self.label_file = label_file
        self.loader = loader
        self.transform = transform
        self.label_map = ['reach','squat','pushup','inline',
                          'hamstrings','lunge','deadbug','stretch']
        self.data= pd.read_csv(self.label_file,header=None)

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.data)

    def map_label_to_int(self,y):
        'The labels need to be integers'
        label_map = {'reach_both': 0,        # the key frames are labeled with the side
                     'squat_both': 1,
                     'inline_left': 2,
                     'inline_right': 2,
                     'lunge_left': 3,
                     'lunge_right': 3,
                     'hamstrings_left': 4,
                     'hamstrings_right': 4,
                     'stretch_left': 5,
                     'stretch_right': 5,
                     'deadbug_left': 6,
                     'deadbug_right': 6,
                     'pushup_both': 7,
                     'reach': 0,            # the video frames don't have information about which side is moving 
                     'squat': 1,
                     'inline': 2,
                     'lunge': 3,
                     'hamstrings': 4,
                     'stretch': 5,
                     'deadbug': 6,
                     'pushup': 7,
                     'label': -1           # label is the placeholder in `videoframe_data_test.txt` for the kaggle frame labels
                    }
        return label_map[y]

    def __getitem__(self,idx):
        'Generates one sample of data'
        path,target = self.data.iloc[idx]
        sample = self.loader(path)
        if self.transform is not None:
            sample = self.transform(sample)
        movement = self.map_label_to_int(target)

        return sample,movement

In [0]:
import os, sys

import torch
import torch.nn as nn
from torch.utils import data
import numpy as np
from skimage import io, transform
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import torchvision.models as models
from PIL import Image
import torch.optim as optim

# Helper functions for loading images.
def pil_loader(path):
    # open path as file to avoid ResourceWarning 
    #(https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')

def accimage_loader(path):
    import accimage
    try:
        return accimage.Image(path)
    except IOError:
        # Potentially a decoding problem, fall back to PIL.Image
        return pil_loader(path)

def default_loader(path):
    from torchvision import get_image_backend
    if get_image_backend() == 'accimage':
        return accimage_loader(path)
    else:
        return pil_loader(path)

# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
#cudnn.benchmark = True

def load_preprocess_data(is_train, is_key_frame, params):
    
    label_file_train, label_file_val, label_file_test = None, None, None
    test_dataset, test_loader = None, None
    
    if is_key_frame:
        label_file_train =  'dataloader_files/keyframe_data_train.txt'
        label_file_val  =  'dataloader_files/keyframe_data_val.txt'
        # NOTE: the kaggle competition test data is only for the video frames, not the key frames
        # this is why we don't have an equivalent label_file_test with keyframes
    else:
        label_file_train = 'dataloader_files/videoframe_data_train.txt'
        label_file_val = 'dataloader_files/videoframe_data_val.txt'
        label_file_test = 'dataloader_files/videoframe_data_test.txt'
        
    mean, std = [0,0,0], [0,0,0]
    if is_key_frame:
      mean = [134.010302198,118.599587912,102.038804945]
      std = [23.5033438916,23.8827343458,24.5498666589]
    else: 
      mean = [133.714058398,118.396875912,102.262895484]
      std = [23.2021839891,23.7064439547,24.3690056102]
    train_dataset = Mds189(label_file_train,loader=default_loader,\
                            transform=transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize(mean, std)
                            ]))
    train_loader = data.DataLoader(train_dataset, **params)

    val_dataset = Mds189(label_file_val,loader=default_loader,\
                          transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize(mean, std)
                          ]))
    val_loader = data.DataLoader(val_dataset, **params)
    
    if not is_key_frame:
        test_dataset = Mds189(label_file_test,loader=default_loader,\
                              transform=transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize(mean, std)
                              ]))
        test_loader = data.DataLoader(test_dataset, **params)
    
    return train_loader, val_loader, test_loader

def modelling(model,is_train,is_key_frame,arch,params,hyperparams):
    train_loader, val_loader, test_loader = \
    load_preprocess_data(is_train,is_key_frame,params)
    train_losses, val_losses = train_val_kagglemodel(model, 
                                                     train_loader,
                                                     val_loader, 
                                                     hyperparams,params)
    return (train_losses, val_losses, arch, params)

def get_current_valacc(model,params):
  model.load_state_dict(torch.load(os.getcwd()+"/"+model.arch))
  model.eval()
  _,val_loader,_ = load_preprocess_data(is_train=False,
                                        is_key_frame=False,
                                        params=params)
  rand_acc,_,_ = get_accuracy(model, val_loader)
  return rand_acc
  
def train_val_kagglemodel(model, train_loader, val_loader, hyperparams,params=None):
    num_epochs,criterion,optimizer = hyperparams["num_epochs"],\
    hyperparams["criterion"],\
    hyperparams["optimizer"]
    
    print('Beginning training...')
    total_step = len(train_loader)
    train_losses, val_losses = [], []
    train_accs, val_accs = [], []
#     scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2, verbose=True)    
    for epoch in range(num_epochs):
        for i, (local_batch,local_labels) in enumerate(train_loader):
            local_ims, local_labels = local_batch.to(device), local_labels.to(device)
            outputs = model.forward(local_ims)
            loss = criterion(outputs, local_labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
            if (i+1) % 10 == 0:
                torch.save(model.state_dict(), os.getcwd()+"/"+model.arch)
                plot_kaggleloss(model.arch,train_losses,val_losses,\
                                (epoch+1, num_epochs, i+1, total_step))
        for i, (local_batch,local_labels) in enumerate(val_loader):
            local_ims, local_labels = local_batch.to(device), local_labels.to(device)
            outputs = model.forward(local_ims)
            loss = criterion(outputs, local_labels)
            val_losses.append(loss.item())
            if (i+1) % 10 == 0:
              plot_kaggleloss(model.arch,train_losses,val_losses)
        print('finished epoch {}, Training Loss: {:.4f}, Validation Loss: {:.4f}'
              .format(epoch+1, train_losses[epoch], val_losses[epoch]))
#         scheduler.step(epoch)
        plot_kaggleloss(model.arch,train_losses,val_losses)
    torch.save(model.state_dict(), os.getcwd()+"/"+model.arch)
    return (train_losses, val_losses)

def get_accuracy(model, loader):
    print('Starting accuracy calculation!!!')
    with torch.no_grad():
      correct = 0
      total = 0
      predicted_list = []
      groundtruth_list = []
      for (local_batch,local_labels) in loader:
          # Transfer to GPU
          local_ims, local_labels = local_batch.to(device), local_labels.to(device)

          outputs = model.forward(local_ims)
          _, predicted = torch.max(outputs.data, 1)
          total += local_labels.size(0)
          predicted_list.extend(predicted)
          groundtruth_list.extend(local_labels)
          correct += (predicted == local_labels).sum().item()
    
    predicted_list = [p.cpu().numpy().tolist() for p in predicted_list]
    groundtruth_list = [p.cpu().numpy().tolist() for p in groundtruth_list]
    acc = correct / total
    print("accuracy",acc)
    return acc,predicted_list,groundtruth_list

def move_val_acc(pl,gt):
    #use pl and gt to produce your confusion matrices

    # view the per-movement accuracy
    label_map = ['reach','squat','inline','lunge','hamstrings','stretch','deadbug','pushup']
    pairs = []
    print(type(pl),type(gt))
    print("pl",pl)
    print("gt",gt)
    for id in range(len(label_map)):
        num = sum([p and g for (p,g) in zip(np.array(pl)==np.array(gt),np.array(gt)==id)])
        denom = sum(np.array(gt)==id)+0.
        acc = num/denom
        move = label_map[id]
        pairs.append((acc,move))
    return pairs

og_map = {'reach_both': 0,        
           'squat_both': 1,
           'inline_left': 2,
           'inline_right': 2,
           'lunge_left': 3,
           'lunge_right': 3,
           'hamstrings_left': 4,
           'hamstrings_right': 4,
           'stretch_left': 5,
           'stretch_right': 5,
           'deadbug_left': 6,
           'deadbug_right': 6,
           'pushup_both': 7,
           'reach': 0,             
           'squat': 1,
           'inline': 2,
           'lunge': 3,
           'hamstrings': 4,
           'stretch': 5,
           'deadbug': 6,
           'pushup': 7,
          }
inv_map = dict((v,k) for k,v in og_map.items())

def test_predict(model,loader):
  print('Starting test predictions!!!')
  with torch.no_grad():
    predicted_list = []
    for (local_batch,local_labels) in loader:
        # Transfer to GPU
        local_ims, local_labels = local_batch.to(device), local_labels.to(device)
        outputs = model.forward(local_ims)
        _, predicted = torch.max(outputs.data, 1)
        predicted_list.extend(predicted)
  names = [format(i, '04')+".jpg" for i in range(len(predicted_list))]
  predicted_list = [inv_map[p.cpu().numpy().tolist()] for p in predicted_list]
  return names, predicted_list

In [0]:
# TRY 1
# class KaggleNN(nn.Module):
#     def __init__(self, arch):
#       super(KaggleNN, self).__init__()
#       self.arch = arch
#       self.conv1 = nn.Conv2d(3,8,11,4)
#       self.mp1 = nn.MaxPool2d(5, 2)
#       self.bn1 = nn.BatchNorm2d(8)
#       self.conv2 = nn.Conv2d(8,16,5,1,2)
#       self.mp2 = nn.MaxPool2d(5, 2)
#       self.bn2 = nn.BatchNorm2d(16)
#       self.conv3 = nn.Conv2d(16,32,3,1,1)
#       self.conv4 = nn.Conv2d(32,64,3,1,1)
#       self.conv5 = nn.Conv2d(64,128,3,1,1)
#       self.mp4 = nn.MaxPool2d(5, 2)        
#       self.fc1 = nn.Linear(5632, 2048)
#       self.fc2 = nn.Linear(2048, 2048)
#       self.fc3 = nn.Linear(2048,8)
#     def forward(self, x):
#       x = self.bn1(F.leaky_relu(self.mp1(self.conv1(x))))
#       x = self.bn2(F.leaky_relu(self.mp2(self.conv2(x))))
#       x = self.conv5(self.conv4(self.conv3(x)))
#       x = self.mp4(x)
#       x = x.view(int(x.size(0)), -1)
#       x = self.fc3(F.dropout(self.fc2(F.dropout(self.fc1(x)))))
#       return x

# TRY 2
# class KaggleNN(nn.Module):
#     def __init__(self, arch):
#       super(KaggleNN, self).__init__()
#       self.arch = arch
#       self.conv1 = nn.Conv2d(3,8,11,4)
#       self.mp1 = nn.MaxPool2d(5, 2)
#       self.bn1 = nn.BatchNorm2d(8)
#       self.conv2 = nn.Conv2d(8,16,5,1,2)
#       self.mp2 = nn.MaxPool2d(5, 2)
#       self.bn2 = nn.BatchNorm2d(16)
#       self.conv3 = nn.Conv2d(16,32,3,1,1)
#       self.bn3 = nn.BatchNorm2d(32)
#       self.conv4 = nn.Conv2d(32,64,3,1,1)
#       self.bn4 = nn.BatchNorm2d(64)
#       self.conv5 = nn.Conv2d(64,128,3,1,1)
#       self.mp3 = nn.MaxPool2d(3, 2)
#       self.bn5 = nn.BatchNorm2d(128)
#       self.fc1 = nn.Linear(7680, 4096)
#       self.fc2 = nn.Linear(4096, 2048)
#       self.fc3 = nn.Linear(2048, 1024)
#       self.fc4 = nn.Linear(1024,8)
#     def forward(self, x):
#       x = self.bn1(F.elu(self.mp1(self.conv1(x))))
#       x = self.bn2(F.elu(self.mp2(self.conv2(x))))
#       x = self.bn3(F.elu(self.conv3(x)))
#       x = self.bn4(F.elu(self.conv4(x)))
#       x = self.bn5(F.elu(self.mp3(self.conv5(x))))
#       x = x.view(int(x.size(0)), -1)
#       x = F.elu(F.dropout(self.fc1(x)))
#       x = F.elu(F.dropout(self.fc2(x)))
#       x = F.elu(F.dropout(self.fc3(x)))
#       x = self.fc4(x)
#       return x

#TRY 3
# class KaggleNN(nn.Module):
#     def __init__(self, arch):
#       super(KaggleNN, self).__init__()
#       self.arch = arch
#       self.conv1 = nn.Conv2d(3,8,11,4)
#       self.mp1 = nn.MaxPool2d(5, 2)
#       self.bn1 = nn.BatchNorm2d(8)
#       self.conv2 = nn.Conv2d(8,16,5,1,2)
#       self.mp2 = nn.MaxPool2d(5, 2)
#       self.bn2 = nn.BatchNorm2d(16)
#       self.conv3 = nn.Conv2d(16,32,3,1,1)
#       self.conv4 = nn.Conv2d(32,64,3,1,1)
#       self.conv5 = nn.Conv2d(64,128,3,1,1)
#       self.mp4 = nn.MaxPool2d(5, 2)        
#       self.fc1 = nn.Linear(5632, 4096)
#       self.fc2 = nn.Linear(4096, 2048)
#       self.fc3 = nn.Linear(2048, 1024)
#       self.fc4 = nn.Linear(1024,8)
#     def forward(self, x):
#       x = self.bn1(F.leaky_relu(self.mp1(self.conv1(x))))
#       x = self.bn2(F.leaky_relu(self.mp2(self.conv2(x))))
#       x = self.conv5(self.conv4(self.conv3(x)))
#       x = self.mp4(x)
#       x = x.view(int(x.size(0)), -1)
#       x = F.dropout(F.leaky_relu(self.fc1(x)))
#       x = F.dropout(F.leaky_relu(self.fc2(x)))
#       x = F.dropout(F.leaky_relu(self.fc3(x)))
#       x = self.fc4(x)
#       return x

#TRY 4-6
# class Flatten(nn.Module):
#     def forward(self, x):
#         return x.view(int(x.size(0)), -1)
# class KaggleNN(nn.Module):
#   def __init__(self, arch):
#     super(KaggleNN, self).__init__()
#     self.arch = arch
#     self.model = nn.Sequential(
#         nn.Conv2d(3, 8, kernel_size=3, padding=1),
#         nn.BatchNorm2d(8),
#         nn.ReLU(inplace=True),
        
#         nn.Conv2d(8,16,3,1),
#         nn.BatchNorm2d(16),
#         nn.ReLU(True),
#         nn.MaxPool2d(kernel_size=2, stride=2),
        
#         nn.Conv2d(16,32,3,1),
#         nn.BatchNorm2d(32),
#         nn.ReLU(True),
        
#         nn.Conv2d(32,64,3,1),
#         nn.BatchNorm2d(64),
#         nn.ReLU(True),
#         nn.MaxPool2d(2,2),
        
#         nn.Conv2d(64,128,3,1),
#         nn.BatchNorm2d(128),
#         nn.ReLU(True),
        
#         nn.Conv2d(128,256,3,1),
#         nn.BatchNorm2d(256),
#         nn.ReLU(True),
#         nn.MaxPool2d(2,2),
        
#         nn.AdaptiveAvgPool2d((7, 7)),
#         Flatten(),
        
#         nn.Linear(12544, 8192),
#         nn.ReLU(True),
#         nn.Dropout(),
        
#         nn.Linear(8192, 4096),
#         nn.ReLU(True),
#         nn.Dropout(),
        
#         nn.Linear(4096, 4096),
#         nn.ReLU(True),
#         nn.Dropout(),     
        
#         nn.Linear(4096, 8),
#     )

#   def forward(self, x):
#     x = self.model(x)
#     return x

#Try 7+
# class KaggleNN(nn.Module):
#     def __init__(self, arch):
#       super(KaggleNN, self).__init__()
#       self.arch = arch
#       self.conv1 = nn.Conv2d(3,8,11,4)
#       self.mp1 = nn.MaxPool2d(5, 2)
#       self.bn1 = nn.BatchNorm2d(8)
#       self.conv2 = nn.Conv2d(8,16,5,1,2)
#       self.mp2 = nn.MaxPool2d(5, 2)
#       self.bn2 = nn.BatchNorm2d(16)
#       self.conv3 = nn.Conv2d(16,32,3,1,1)
#       self.bn3 = nn.BatchNorm2d(32)
#       self.conv4 = nn.Conv2d(32,64,3,1,1)
#       self.bn4 = nn.BatchNorm2d(64)
#       self.conv5 = nn.Conv2d(64,128,3,1,1)
#       self.bn5 = nn.BatchNorm2d(128)
#       self.mp3 = nn.MaxPool2d(5, 2)     
#       self.bn6 = nn.BatchNorm1d(2048)
#       self.bn7 = nn.BatchNorm1d(2048)
#       self.fc1 = nn.Linear(5632, 2048)
#       self.fc2 = nn.Linear(2048, 2048)
#       self.fc3 = nn.Linear(2048,8)
#       self.prelu1 = nn.PReLU()
#       self.prelu2 = nn.PReLU()
#       self.prelu3 = nn.PReLU()
#       self.prelu4 = nn.PReLU()
#       self.prelu5 = nn.PReLU()
#       self.prelu6 = nn.PReLU()
#       self.prelu7 = nn.PReLU()
#     def forward(self, x):
#       x = self.mp1(self.bn1(self.prelu1(self.conv1(x))))
#       x = self.mp2(self.bn2(self.prelu2(self.conv2(x))))
#       x = self.bn3(self.prelu3(self.conv3(x)))
#       x = self.bn4(self.prelu4(self.conv4(x)))
#       x = self.bn5(self.prelu5(self.conv5(x)))
#       x = self.mp3(x)
#       x = x.view(int(x.size(0)), -1)
#       x = F.dropout(self.bn6(self.prelu6(self.fc1(x))))
#       x = F.dropout(self.bn7(self.prelu7(self.fc2(x))))
#       x = self.fc3(x)
#       return x

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class KaggleNN(nn.Module):
    def __init__(self, arch):
      super(KaggleNN, self).__init__()
      self.arch = arch
      self.conv1 = nn.Conv2d(3,8,11,4)
      self.mp1 = nn.MaxPool2d(5, 2)
      self.bn1 = nn.BatchNorm2d(8)
      self.conv2 = nn.Conv2d(8,16,5,1,2)
      self.mp2 = nn.MaxPool2d(5, 2)
      self.bn2 = nn.BatchNorm2d(16)
      self.conv3 = nn.Conv2d(16,32,3,1,1)
      self.bn3 = nn.BatchNorm2d(32)
      self.conv4 = nn.Conv2d(32,64,3,1,1)
      self.bn4 = nn.BatchNorm2d(64)
      self.conv5 = nn.Conv2d(64,128,3,1,1)
      self.bn5 = nn.BatchNorm2d(128)
      self.mp3 = nn.MaxPool2d(5, 2)     
      self.bn6 = nn.BatchNorm1d(2048)
      self.bn7 = nn.BatchNorm1d(2048)
      self.fc1 = nn.Linear(5632, 2048)
      self.fc2 = nn.Linear(2048, 2048)
      self.fc3 = nn.Linear(2048,8)
      self.prelu1 = nn.PReLU()
      self.prelu2 = nn.PReLU()
      self.prelu3 = nn.PReLU()
      self.prelu4 = nn.PReLU()
      self.prelu5 = nn.PReLU()
      self.prelu6 = nn.PReLU()
      self.prelu7 = nn.PReLU()
    def forward(self, x):
      x = self.mp1(self.bn1(self.prelu1(self.conv1(x))))
      x = self.mp2(self.bn2(self.prelu2(self.conv2(x))))
      x = self.bn3(self.prelu3(self.conv3(x)))
      x = self.bn4(self.prelu4(self.conv4(x)))
      x = self.bn5(self.prelu5(self.conv5(x)))
      x = self.mp3(x)
      x = x.view(int(x.size(0)), -1)
      x = F.dropout(self.bn6(self.prelu6(self.fc1(x))))
      x = F.dropout(self.bn7(self.prelu7(self.fc2(x))))
      x = self.fc3(x)
      return x

In [0]:
def weights_init(m):
  if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
      torch.nn.init.xavier_uniform_(m.weight)
      if m.bias is not None:
          torch.nn.init.zeros_(m.bias)

In [0]:
import pandas as pd
import numpy as np
import datetime

def preds_to_csv(names,y):
    df = pd.DataFrame({'Id': names, 'Category': y})
    df.set_index('Id',inplace=True)
    df.to_csv(str(str(datetime.datetime.now().time())+'_submission.csv'))
    print("saved predictions")

In [0]:
!ls

# 4g

In [0]:
import adabound 

is_train = True
is_key_frame = False
arch = "pytorch_kaggle1.5"
params = {'batch_size': 64,'shuffle': True,'num_workers': 4}
hyperparams = {"num_epochs":3,"criterion":nn.CrossEntropyLoss()}
model = KaggleNN(arch).to(device)
model.apply(weights_init)
hyperparams["optimizer"] = adabound.AdaBound(model.parameters(), lr=1e-3, final_lr=0.1)

In [0]:
mod_data = modelling(model,is_train,is_key_frame,arch,params,hyperparams)

In [0]:
models_data.append(mod_data)
data_track(models_data,save=True)

In [0]:
model = KaggleNN(arch).to(device)
model.load_state_dict(torch.load(os.getcwd()+"/"+arch))
model.eval()
val_params = {'batch_size': 64,'shuffle': False,'num_workers': 4}
_,val_loader,_ = load_preprocess_data(is_train=False,
                                         is_key_frame=False,
                                         params=val_params)
rand_acc,rand_predicted_list,rand_groundtruth_list = get_accuracy(model, val_loader)
rand_pairs = move_val_acc(rand_predicted_list, rand_groundtruth_list)

In [0]:
accs = [[p[0] for p in rand_pairs]+[rand_acc]]
cols = ['reach','squat','inline','lunge','hamstrings','stretch','deadbug','pushup',"overall"]
df = pd.DataFrame(accs,columns=cols)
print("accuracies")
df

In [0]:
cm = confusion_matrix(rand_groundtruth_list, 
                 rand_predicted_list)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
fig, ax = plt.subplots()
classes = ['reach','squat','inline','lunge','hamstrings','stretch','deadbug','pushup']
im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
ax.figure.colorbar(im, ax=ax)
ax.set(xticks=np.arange(cm.shape[1]),
       yticks=np.arange(cm.shape[0]),
       xticklabels=classes, yticklabels=classes,
       title="confusion matrix for Kaggle model",
       ylabel='True label',
       xlabel='Predicted label');
plt.xticks(rotation=90);

In [0]:
_,_,test_loader = load_preprocess_data(is_train=False,
                                          is_key_frame=False,
                                          params=val_params)
names, kaggle_preds = test_predict(model,test_loader)
preds_to_csv(names, kaggle_preds)